In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\risla\OneDrive\Desktop\EDA raw\data.csv")
df.head()

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Insurance,Groceries,Transport,...,Desired_Savings,Disposable_Income,Potential_Savings_Groceries,Potential_Savings_Transport,Potential_Savings_Eating_Out,Potential_Savings_Entertainment,Potential_Savings_Utilities,Potential_Savings_Healthcare,Potential_Savings_Education,Potential_Savings_Miscellaneous
0,44637.249636,49,0,Self_Employed,Tier_1,13391.174891,0.000000,2206.490129,6658.768341,2636.970696,...,6200.537192,11265.627707,1685.696222,328.895281,465.769172,195.151320,678.292859,67.682471,0.000000,85.735517
1,26858.596592,34,2,Retired,Tier_2,5371.719318,0.000000,869.522617,2818.444460,1543.018778,...,1923.176434,9676.818733,540.306561,119.347139,141.866089,234.131168,286.668408,6.603212,56.306874,97.388606
2,50367.605084,35,1,Student,Tier_3,7555.140763,4612.103386,2201.800050,6313.222081,3221.396403,...,7050.360422,13891.450624,1466.073984,473.549752,410.857129,459.965256,488.383423,7.290892,106.653597,138.542422
3,101455.600247,21,0,Self_Employed,Tier_3,15218.340037,6809.441427,4889.418087,14690.149363,7106.130005,...,16694.965136,31617.953615,1875.932770,762.020789,1241.017448,320.190594,1389.815033,193.502754,0.000000,296.041183
4,24875.283548,52,4,Professional,Tier_2,4975.056710,3112.609398,635.907170,3034.329665,1276.155163,...,1874.099434,6265.700532,788.953124,68.160766,61.712505,187.173750,194.117130,47.294591,67.388120,96.557076


**Fix Datatypes**

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Income                           20000 non-null  float64
 1   Age                              20000 non-null  int64  
 2   Dependents                       20000 non-null  int64  
 3   Occupation                       20000 non-null  object 
 4   City_Tier                        20000 non-null  object 
 5   Rent                             20000 non-null  float64
 6   Loan_Repayment                   20000 non-null  float64
 7   Insurance                        20000 non-null  float64
 8   Groceries                        20000 non-null  float64
 9   Transport                        20000 non-null  float64
 10  Eating_Out                       20000 non-null  float64
 11  Entertainment                    20000 non-null  float64
 12  Utilities         

In [10]:
df['Occupation']=df['Occupation'].astype('category')
df['City_Tier']=df['City_Tier'].astype('category')


# Converted categorical columns to category type
# for memory efficiency and correct analysis.


 **Standardize Categorical Variables**

In [11]:
df['City_Tier'].unique()
df['Occupation'].unique()


['Self_Employed', 'Retired', 'Student', 'Professional']
Categories (4, object): ['Professional', 'Retired', 'Self_Employed', 'Student']

**Remove Duplicates**

In [12]:
df.duplicated().sum()

# no duplicate values


np.int64(0)

**Handle Missing Values**


In [13]:
df.isnull().sum()

# Dataset contains no missing values; no imputation required.

Income                             0
Age                                0
Dependents                         0
Occupation                         0
City_Tier                          0
Rent                               0
Loan_Repayment                     0
Insurance                          0
Groceries                          0
Transport                          0
Eating_Out                         0
Entertainment                      0
Utilities                          0
Healthcare                         0
Education                          0
Miscellaneous                      0
Desired_Savings_Percentage         0
Desired_Savings                    0
Disposable_Income                  0
Potential_Savings_Groceries        0
Potential_Savings_Transport        0
Potential_Savings_Eating_Out       0
Potential_Savings_Entertainment    0
Potential_Savings_Utilities        0
Potential_Savings_Healthcare       0
Potential_Savings_Education        0
Potential_Savings_Miscellaneous    0
d

##### **Outiler detection**



In [14]:
num_cols=df.select_dtypes(include='number').columns
Q1=df[num_cols].quantile(0.25)
Q3=df[num_cols].quantile(0.75)
IQR=Q3-Q1
lower=Q1-1.5*IQR
upper=Q3+1.5*IQR
outliers_count=((df[num_cols]<lower)|(df[num_cols]>upper)).sum()
outliers_count

# Outliers were identified using the IQR method.
# Since extreme values reflect real financial behaviour rather than errors, 
# no rows were removed or capped.


Income                             1254
Age                                   0
Dependents                            0
Rent                               1320
Loan_Repayment                     1880
Insurance                          1375
Groceries                          1227
Transport                          1261
Eating_Out                         1325
Entertainment                      1324
Utilities                          1291
Healthcare                         1294
Education                          1159
Miscellaneous                      1348
Desired_Savings_Percentage          789
Desired_Savings                    1303
Disposable_Income                  1331
Potential_Savings_Groceries        1401
Potential_Savings_Transport        1395
Potential_Savings_Eating_Out       1431
Potential_Savings_Entertainment    1453
Potential_Savings_Utilities        1383
Potential_Savings_Healthcare       1417
Potential_Savings_Education        1347
Potential_Savings_Miscellaneous    1471


### **Feature Engineering**

New financial indicators were created to simplify behavioural analysis, including total expense, savings rate, essential expense, lifestyle expense, and lifestyle ratio.

**Total Expense**

In [15]:
expense_cols = [
    'Rent','Loan_Repayment','Insurance','Groceries','Transport',
    'Eating_Out','Entertainment','Utilities','Healthcare','Education','Miscellaneous'
]

df['Total_Expense'] = df[expense_cols].sum(axis=1)



**Savings Rate**

In [16]:
df['Savings_Rate'] = df['Disposable_Income'] / df['Income']


**Lifestyle Spending**

In [17]:
lifestyle_cols = ['Eating_Out','Entertainment','Miscellaneous']

df['Lifestyle_Expense'] = df[lifestyle_cols].sum(axis=1)


**Necessity Spending**

In [18]:
necessity_cols = ['Rent','Groceries','Utilities','Healthcare','Education','Transport','Insurance','Loan_Repayment']

df['Essential_Expense'] = df[necessity_cols].sum(axis=1)


**Extra Savings**

In [19]:
df['Extra_Savings']=df['Disposable_Income']-df['Desired_Savings']

**Income Segmenet**

In [20]:
df['Income_Segment']=pd.cut(df['Income'],bins=[0,20000,40000,70000,float('inf')],labels=['low','middle','high','very_high'])
df['Income_Segment'].value_counts()

Income_Segment
middle       6683
low          6124
high         4251
very_high    2942
Name: count, dtype: int64

## Feature Engineering Summary

|   | Feature | Description |
|---|---------|-------------|
| 1 | Total_Expense | Sum of all monthly expense columns |
| 2 | Savings_Rate | Disposable Income / Income |
| 3 | Lifestyle_Expense | Eating Out + Entertainment + Miscellaneous |
| 4 | Essential_Expense | Rent + Groceries + Utilities + Healthcare + Transport + Insurance + Loan |
| 5 | Extra_Savings | Disposable Income - Desired Savings |
| 6 | Income_Segment | Income categorized into Low/Middle/High/Very High |

In [23]:
# to save

df.to_csv("cleaned_day2.csv",index=False)
print ("saved")


saved
